In [ ]:
import librosa
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
import scipy.signal as signal
from typing import List, Tuple, Callable
from notes import note

In [ ]:
def clip(a: np.array, threshold: float, both: bool = True) -> np.array:
    max = np.max(a)
    a = a / max
    a = np.where(a < threshold, a, threshold)
    if both:
        a = np.where(a > -threshold, a, -threshold)
    clipped = a * max
    return clipped

In [ ]:
def delay(a: np.array, amount: int) -> np.array:
    return a + 0.5* np.concatenate([np.zeros(amount), a])[:-amount]

In [ ]:
def normalize(a: np.array) -> np.array:
    return a / np.max(a)

In [ ]:
def get_signal(frequency: float, function: Callable = np.sin, part: float = 1, samplerate: int = 44100) -> np.array:
    n = samplerate
    t = np.linspace(0, 1, samplerate)
    wave = function(t*2*np.pi*frequency)
    return wave[:int(n*part)]

In [ ]:
def chord(notes: List[str], function: Callable = np.sin, part: float = 1, samplerate: int = 44100) -> np.array:
    n = samplerate
    wave = np.zeros(samplerate)
    for note_name in notes:
        wave = wave + get_signal(note[note_name], function)
    return normalize(wave)[:int(n*part)]

In [ ]:
def melody(notes: List[Tuple[str, float]], function: Callable = np.sin, samplerate: int = 44100) -> np.array:
    """The notes should be a list of tuples. If only a single note (str) is found, it is converted to (str, 1)."""
    convert = lambda n: (n, 1) if type(n) == str else n
    notes = [convert(n) for n in notes]
    melody = np.concatenate([get_signal(note[n], part=p) for n, p in notes])
    return melody

In [ ]:
def play(a: np.array, samplerate: int = 44100, volume: float = 0.2, repeat: int = 1):
    wave = np.tile(normalize(a)*volume, repeat)
    return ipd.Audio(wave, rate=samplerate, autoplay=True, normalize=False)

In [ ]:
c = chord(["C2", "E4", "G4"])
am = chord(["C2", "A3",  "E4"])
f = chord(["F3", "A4", "C4"])
g = chord(["G3", "B4", "D4"])
left = np.concatenate([c, am, f, g])

c  = chord(["C2"], signal.square)*0.3 + 0*chord(["C2"], signal.square) + chord(["C2"])
am = chord(["E2"], signal.square)*0.3 + 0*chord(["E2"], signal.square) + chord(["E2"])
f  = chord(["F2"], signal.square)*0.3 + 0*chord(["F2"], signal.square) + chord(["F2"])
g  = chord(["D2"], signal.square)*0.3 + 0*chord(["D2"], signal.square) + chord(["D2"])
right = np.concatenate([c, am, f, g]) * 0.4

left, right = (left*0.9 + right*0.2), (left*0.1 + right*0.5)
left, right = clip(left, 0.3)*0.5 + left*0.3, 0.5*right + clip(right, 0.1) * 0.5

song = np.c_[left, right]
# song = np.concatenate([song, clip(song, 0.9), clip(song, 0.8), clip(song, 0.7), clip(song, 0.6), clip(song, 0.5), clip(song, 0.4), clip(song, 0.3), clip(song, 0.2), clip(song, 0.1), clip(song, 0.05)])
fig, axs = plt.subplots(2, 1, figsize=(64,16))
plt.ylim((-1, 1))
axs[0].plot(left)
axs[1].plot(right)
play([left, right], repeat=6, volume=0.08)

In [ ]:
vol1 = np.sum(right)
clipped = clip(right, 0.1)
vol2 = np.sum(clipped)
print(vol1, vol2)

In [ ]:
librosa.output.write_wav("out/song.wav", song.T, 44100)

In [ ]:
tooth = signal.sawtooth(np.linspace(0, 2*np.pi, 1000))
clipped = clip(tooth, 0.1, both=True)
plt.plot(clipped)